In [37]:
import numpy as np
import pandas as pd
import h5py
from sklearn.preprocessing import scale, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model

In [38]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [39]:
_Y = pd.read_hdf('matched_output_data.hdf5', key = 'output1')
_X = pd.read_hdf('matched_input_data.hdf5', key = 'input1')

In [40]:
#TODO should do this in one of the cleaning NBs
_Y =  _Y[~np.any(np.isnan(_X), axis = 1)]
_X =  _X[~np.any(np.isnan(_X), axis = 1)]

In [41]:
chosen_x_columns = ['raICRS', 'decICRS', 'g_mag', 'i_mag', u'majorAxis', 'minorAxis',
       'positionAngle', 'r_mag', 'redshift', 'sindex', 'u_mag',
       'y_mag', 'z_mag']
X = _X[chosen_x_columns]
X['majorAxis'] = np.log10(X['majorAxis'])
X['minorAxis'] = np.log10(X['minorAxis'])

/afs/slac.stanford.edu/u/ki/swmclau2/.local/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/afs/slac.stanford.edu/u/ki/swmclau2/.local/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [42]:
#chosen_y_columns = ['modelfit_CModel_ellipse_xx']#,'modelfit_CModel_ellipse_yy','modelfit_CModel_ellipse_xy']
chosen_y_columns = ['coord_ra', 'coord_dec','cmodelMag','modelfit_CModel_flux',\
                   'modelfit_CModel_ellipse_xx','modelfit_CModel_ellipse_yy','modelfit_CModel_ellipse_xy']


Y = _Y[chosen_y_columns]

#Y['modelfit_CModel_ellipse_xx'] = np.log10(Y['modelfit_CModel_ellipse_xx'])
#Y['modelfit_CModel_ellipse_yy'] = np.log10(Y['modelfit_CModel_ellipse_yy'])
#Y['modelfit_CModel_ellipse_xy'] = np.log10(Y['modelfit_CModel_ellipse_xy'])

/afs/slac.stanford.edu/u/ki/swmclau2/.local/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/afs/slac.stanford.edu/u/ki/swmclau2/.local/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/afs/slac.stanford.edu/u/ki/swmclau2/.local/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [43]:
X.columns

Index([u'raICRS', u'decICRS', u'g_mag', u'i_mag', u'majorAxis', u'minorAxis',
       u'positionAngle', u'r_mag', u'redshift', u'sindex', u'u_mag', u'y_mag',
       u'z_mag'],
      dtype='object')

In [44]:
np.c_[[X[col] for col in X.columns]].T.shape

(524736, 13)

In [45]:
x_scaler = StandardScaler()
X_white = x_scaler.fit_transform(np.c_[[X[col] for col in X.columns]].T)

In [46]:
y_scaler = StandardScaler()
Y_white = y_scaler.fit_transform(np.c_[[Y[col] for col in Y.columns]].T)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_white, Y_white, test_size=0.3, random_state=42)

In [ ]:
regr = RandomForestRegressor(n_estimators = 50, n_jobs = 2)
#regr = linear_model.LinearRegression()
#regr = linear_model.Lasso(alpha = 1.0)

In [ ]:
regr.fit(X_train, Y_train)

In [ ]:
regr.score(X_train, Y_train)

In [ ]:
regr.score(X_test, Y_test)

In [ ]:
for name, imp in sorted(zip(X.columns, regr.feature_importances_), key = lambda x: x[1], reverse = True):
    print name, imp

In [ ]:
Y_pred = regr.predict(X_test)

In [ ]:
X.columns

In [ ]:
X_test.shape

In [ ]:
print Y.columns

In [ ]:
idx = 4
sns.jointplot(np.log10(y_scaler.inverse_transform(Y_test)[:,idx]),\
              np.log10(y_scaler.inverse_transform(Y_pred)[:,idx]),\
              kind = 'scatter', alpha = 0.1).set_axis_labels("log Predicted %s"%Y.columns[idx],"log True %s"%Y.columns[idx], )#,stat_func = None)

In [ ]:
plt.scatter(X['majorAxis'], X['minorAxis'])

In [ ]:
for i in xrange(X_test.shape[1]):
    print X.columns[i]
    plt.scatter(x_scaler.inverse_transform(X_test)[:,i], np.log10(MSE), alpha = 0.1)
    plt.show()